In [1]:
%pip install -Uq upgini
%pip install -Uq shap
import pandas as pd
import numpy as np
from os.path import exists

     |████████████████████████████████| 91 kB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 23.6 MB/s 
     |████████████████████████████████| 76.6 MB 19 kB/s 
     |████████████████████████████████| 12.2 MB 2.6 MB/s 
     |████████████████████████████████| 2.0 MB 13.9 MB/s 
     |████████████████████████████████| 575 kB 4.3 MB/s 


In [ ]:
df_path = "https://github.com/upgini/upgini/raw/main/notebooks/senate_elections/us_zip_elections.parquet"
df = pd.read_parquet(df_path)

In [6]:
train=df.loc[df['election_date']<'2022-11-08']
train=train.drop_duplicates(subset=['postal_code'], keep='last')
test=df.loc[df['election_date']=='2022-11-08']

In [9]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import RuntimeParameters,CVType

enricher = FeaturesEnricher(
    search_keys={
        'country': SearchKey.COUNTRY,
        'postal_code': SearchKey.POSTAL_CODE,
        'election_date':SearchKey.DATE,      
    }, 
    cv=CVType.time_series,
)
enricher.fit(train[['country', 'postal_code', 'population', 'density','last_result',  'population_county','last_result_share', 'election_date', 'last_result_party']], train.target_result_party,
             eval_set=[(test[['country', 'postal_code', 'population', 'density','last_result',  'population_county','last_result_share', 'election_date', 'last_result_party']], test.target_result_party)],
             calculate_metrics=True)

Detected task type: ModelTaskType.BINARY


Column name,Status,Description
election_date,All valid,All values in this column are good to go
postal_code,All valid,All values in this column are good to go
target,All valid,All values in this column are good to go
country,All valid,All values in this column are good to go


Running search request with search_id=f2ef25bb-279e-4195-9638-2c09d2976cc2
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done


42 relevant feature(s) found with the search keys: ['country', 'postal_code', 'election_date']


,,last_result_share,0.424737,100.000000,numerical,
,,last_result,0.147414,100.000000,numerical,
,,population_county,0.095969,100.000000,numerical,
Upgini,Public/Comm. shared,f_location_country_postal_latitude_2e1eae46,0.041621,100.000000,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_daylight_time_bea3cf0a,0.037600,21.444740,numerical,Free
Upgini,Public/Comm. shared,f_location_country_postal_longitude_585c92dc,0.032560,100.000000,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_prcp_4d9ed1e1,0.020857,99.771653,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_delta_to_avg_snow_8bd64d2e,0.016698,97.970692,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_tobs_210f1c58,0.012317,91.723919,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_daylight_time_a79bd1f1,0.011364,78.555260,numerical,Free
Upgini,Public/Comm. shared,f_weather_country_date_postal_delta_to_avg_prcp_69cb1eca,0.009370,99.771653,numerical,Free


Calculating metrics...
Done

Quality metrics


,match_rate,baseline roc_auc,enriched roc_auc,uplift
,,,,
train,100.0,0.915854,0.922462,0.006608
eval 1,100.0,0.950074,0.956784,0.006710


In [10]:
df2=enricher.transform(test, max_features=10)

Column name,Status,Description
election_date,All valid,All values in this column are good to go
postal_code,All valid,All values in this column are good to go
country,All valid,All values in this column are good to go


Running search request with search_id=92e10133-6507-499c-baf2-3ccdd934b08d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
Done

Done


In [11]:
df2.groupby(['target_result_party'])['f_weather_country_date_postal_delta_to_avg_prcp_69cb1eca'].mean()

target_result_party
0     9.072786
1    12.687661
Name: f_weather_country_date_postal_delta_to_avg_prcp_69cb1eca, dtype: float64